In [4]:
import RMC
import numpy as np
import GPy
X0 =np.random.normal(5,np.sqrt(0.5),10000)
process = RMC.simulate.OU(X0,96,10000,24,[1],[5],[1])
running_cost = RMC.costfunctions.L2()
final_cost = RMC.costfunctions.final_SOCcontraint(0,0)
parameters = (2,8,0.95,0.05)
batch_size = 30
value_kernel= GPy.kern.Matern52
normalize_v = True
policy_kernel = GPy.kern.Matern32
normalize_policy = True
hybrid_solution = RMC.model.HybridControl(600,process,running_cost,final_cost,parameters,batch_size,\
                                          value_kernel,normalize_v,policy_kernel,normalize_policy)

In [5]:
hybrid_solution.solve()

KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: 

In [10]:
5-2.5*hybrid_solution.process.std_vec

array([3.24139888, 3.18679476, 3.14446685, 3.12754761, 3.11523717,
       3.10551907, 3.10178616, 3.09104663, 3.09992653, 3.10574635,
       3.11541328, 3.11018836, 3.09514257, 3.12738807, 3.11239709,
       3.09864536, 3.09352567, 3.10120782, 3.12298388, 3.14046745,
       3.14424389, 3.11835677, 3.11378861, 3.10600848, 3.10340127,
       3.0959649 , 3.08760504, 3.09724183, 3.11430448, 3.10012848,
       3.08620167, 3.10525511, 3.10715097, 3.10236254, 3.10627147,
       3.1068399 , 3.10879134, 3.08865896, 3.08850195, 3.11170312,
       3.10413281, 3.11804538, 3.11412996, 3.0992559 , 3.0938489 ,
       3.09109403, 3.10534332, 3.10025376, 3.10097535, 3.0856129 ,
       3.09535392, 3.08362668, 3.09683742, 3.09510336, 3.09719461,
       3.09706343, 3.09056272, 3.08956882, 3.09392692, 3.09018455,
       3.09390008, 3.09502195, 3.13620522, 3.12202901, 3.1112674 ,
       3.09393289, 3.08917134, 3.10207714, 3.11618979, 3.11024907,
       3.12337674, 3.11425633, 3.13565218, 3.14041105, 3.13201

In [11]:
X_range = np.arange(3.5,6.5+0.05,0.05)
I_range  = np.arange(0.4,7.6,0.05)
B_range_t0 = np.zeros((len(I_range),len(X_range)))
B_range_t0_cap = np.zeros((len(I_range),len(X_range)))
import matplotlib.pyplot as plt
X, Y = np.meshgrid(X_range, I_range)
for i in range(len(I_range)):
    charges = I_range[i] * np.ones(len(X_range))
    LB = np.maximum(-2, (0.4-charges)/0.25);UB = np.minimum(2, (7.6-charges)/0.25)
    inp = np.column_stack((X_range,charges))
    B_range_t0[i,:] = hybrid_solution.policies[50].predict(inp)[0].flatten()
    B_range_t0_cap[i,:] = B_range_t0[i,:]
    idx = np.where(B_range_t0_cap[i,:]>0)
    upper_bound = np.maximum(X_range[idx],0)
    B_range_t0_cap[i,idx] =  np.minimum(B_range_t0_cap[i,idx] ,upper_bound)
    B_range_t0_cap[i,:] = np.maximum(LB,np.minimum(B_range_t0_cap[i,:],UB))

In [1]:

import matplotlib.pyplot as plt
X, Y = np.meshgrid(X_range, I_range)

fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(121,projection= "3d")
ax.plot_surface(X, Y, B_range_t0_cap, cmap='cool', alpha=0.8)
ax.view_init(30, 110)

NameError: name 'np' is not defined

In [14]:
hybrid_solution.policies[:50] = hybrid_solution.policies[50]

50

In [15]:

process = RMC.simulate.OU(5,96,10000,24,[1],[5],[1])
total_cost = np.zeros(10000)
cur_I = np.ones(10000) *1

for i in range(96):
    cur_X = process.sim_trajectories[:,i]
    GP_inp = np.column_stack((cur_X,cur_I))
    B = hybrid_solution.policy_maps[i].predict(GP_inp)[0].flatten()
    pos_Bs = np.where(B>0)
    upper_bound = np.maximum(cur_X[pos_Bs]-5,0)
    B[pos_Bs] =  np.minimum(B[pos_Bs] ,upper_bound)
    LB = np.maximum(hybrid_solution.Bmin,hybrid_solution.charging_eff*(hybrid_solution.Ilb- cur_I)/hybrid_solution.dt)
    UB = np.minimum(hybrid_solution.Bmax,(hybrid_solution.Iub-cur_I)/(hybrid_solution.charging_eff*hybrid_solution.dt))
    B = np.maximum(LB,np.minimum(B,UB))
    cur_I = cur_I + B * ( (B>0) * hybrid_solution.charging_eff + (B<0) *1/hybrid_solution.charging_eff) * hybrid_solution.dt
    total_cost = total_cost+ (cur_X- B-5)**2 * process.dt

AttributeError: 'numpy.float64' object has no attribute 'predict'

In [14]:
np.mean(total_cost)

2.5898260685273997